In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
# İBB verisinin csv olarak kaydedilmiş hali.
datafile = "./trafik-indeks-raporu.csv"
# 2019 yılı içindeki resmi tatilleri içeren dosya.
resmitatilfile = "./Resmi-Tatiller.csv"
# 2019 yılı içindeki resmi tatiller hariç okul tatillerini içeren dosya.
okultatilfile = "./Okul-Tatilleri.csv"

In [3]:
df = pd.read_csv(datafile, sep=";")

# İsimeri basitleştirelim.
df.rename(columns={'Trafik İndeksi': 'Trafik', 'Trafik İndeksi Tarihi': 'Zaman Damgası'}, inplace=True)

# Tarihler iki değişik formatta. Çoğu "d.mm.yyyy  ss:dd" formatında 
# (Excel'de saniyesi de var ama CSV'ye atarken onları atlamış).
# Geri kalanı "yyyy-mm-dd ss:dd:nn.nnn" formatında. O yüzden to_datetime diyerek çevirmek lazım.
# to_datetime birden çok formatı çevirebiliyor, ama dayfirst=True demezsek biraz kafası
# karışabiliyor bu veriyle:)
df['Zaman Damgası'] = pd.to_datetime(df['Zaman Damgası'], dayfirst=True)

# Bazı yerlerde 255 değeri var. Yaklaşık 500 satır kadar. Bunlar belli ki hatalı veriler çünkü
# bunlar dışında max değer 81. Dolayısıyla bunları np.nan olarak değiştirelim.
df.replace({255: np.nan}, inplace=True)

# Şu andaki endeks anlamlı bir endeks değil, ID'de aslında gerekli değil.
# Gerçek ve yararlı olacak endeks gözlem zamanı. Bu yüzdem endeksi de 'Zaman Damgası' olarak değiştirelim.
df.set_index('Zaman Damgası', inplace=True)
df.drop('ID', axis=1, inplace=True)

In [4]:
# Arada np.nan yaptığımız 255'lerin yerine zaman bazlı interplotationla tahmini değerler koyalım.
df = df.interpolate(method='time')

In [5]:
# Sadece 2019'a bakalım. Slice ederken ilk değer dahil, son değer hariç:)
df.sort_index(inplace=True)
df = df.loc[datetime.date(year=2019, month=1, day=1):datetime.date(year=2020, month=1, day=1)]

In [6]:
# Veriyi daha kolay kullanılır hale getirmek için ay, haftanın günü, ssat ve günden bağımsız zaman
# kolonlarını ekleyelim.

# Tam zamanın endekste kalması bazı grafikleri zorlaştıracak. Onu normal bir kolon haline getirelim.
df.reset_index(inplace=True)
df.rename(columns={'index': 'Zaman Damgası'}, inplace=True)

# Tarihi (saatsiz) kendi sütununa koyalım.
df['Tarih'] = df.apply(lambda tr: pd.datetime(tr['Zaman Damgası'].year, 
                                             tr['Zaman Damgası'].month,
                                             tr['Zaman Damgası'].day), axis=1)

# Bazı veriler "yyyy-mm-dd ss:dd:nn.nnn" formatındaydı. Onları datetime'a çevirirken milisaniye
# detayını da alıyoruz. Gereksiz ve çirkin. Bu yüzden milisaniyelerden kurtarıp tüm zamanları
# ss:dd:nn formatına çevirelim (string olacak bunlar mecburen).
def resetms(tarih):
    saat = tarih.time()
    newtime = datetime.time(saat.hour, saat.minute, saat.second)
    return newtime
df['Saat'] = df.apply(lambda tr: resetms(tr['Zaman Damgası']), axis=1)

# Ayları sayı olarak değil Türkçe ay adlarına çevirelim ve kendi sütununa koyalım.
df['Ay'] = df.apply(lambda tr: tr['Zaman Damgası'].month, axis=1) 
df['Ay'] = pd.Categorical(df['Ay'], 
        categories=list(range(1,13)),
        ordered=True)
df['Ay'] = df['Ay'].cat.rename_categories(
    ['Ocak', 'Şubat', 'Mart', 'Nisan', 'Mayıs', 'Haziran', 
    'Temmuz', 'Ağustos', 'Eylül', 'Ekim', 'Kasım', 'Aralık'])

# Haftanın günlerini de çevirelim ve kendi sütununa koyalım.
df['Gün'] = df.apply(lambda tr: tr['Zaman Damgası'].day_name(), axis=1) 
df['Gün'] = pd.Categorical(df['Gün'], 
        categories=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"],
        ordered=True)
df['Gün'] = df['Gün'].cat.rename_categories(
    ["Pazartesi", "Salı", "Çarşamba", "Perşembe", "Cuma", "Cumartesi", "Pazar"])

# Ayın günlerini kendi sütununa koyalım..
df['Ayın Günü'] = df.apply(lambda tr: tr['Tarih'].day, axis=1)

# Zamanın saat kısmını (dakikasız ve saniyesiz)  kendi sütununa koyalım.
df['Tam Saat'] = df.apply(lambda tr: tr['Zaman Damgası'].hour, axis=1)

# Tatiileri de ekleyelim.
resmitatil = pd.read_csv(resmitatilfile)
resmitatil['Resmi Tatil'] = pd.to_datetime(resmitatil['Resmi Tatil'])
resmitatil_lst = list(resmitatil['Resmi Tatil'])

okultatil = pd.read_csv(okultatilfile)
okultatil['Okul Tatilleri'] = pd.to_datetime(okultatil['Okul Tatilleri'])
okultatil_lst = list(okultatil['Okul Tatilleri'])

df['Resmi Tatil'] = df.apply(lambda tr: tr['Tarih'] in resmitatil_lst, axis=1)
df['Okul Tatili'] = df.apply(lambda tr: tr['Tarih'] in okultatil_lst, axis=1)
df['Haftasonu'] = df.apply(lambda tr: tr['Gün'] in ["Cumartesi", "Pazar"], axis=1)


In [7]:
# Temizlenmiş veri bir kenarda dursun.
df.to_csv("Istanbul_Trafik_Endeksleri_(2019)_(Temiz).csv", index=False)
df.to_pickle("Istanbul_Trafik_Endeksleri_(2019)_(Temiz).pkl")